In [49]:
import numpy as np
import pandas as pd
from pandas.tseries.offsets import BDay
import ta
import matplotlib.pyplot as plt
import vnstock as vn
from datetime import timedelta
from vnstock3 import Vnstock

# Setting

In [50]:
rf = {'2019': 0.0451, '2020': 0.0286, '2021':0.023,'2022': 0.0335,'2023':0.0321}
rf = pd.Series(rf)
rf_cal = rf.mean()
rf_2022 = 0.0335

In [51]:
pd.set_option('display.max_columns', None) 

In [52]:
RSI_PERIOD = 14
RSI_OVERSOLD = 30
RSI_OVERBOUGHT = 70
MACD_SLOW_PERIOD = 26
MACD_FAST_PERIOD = 12
MACD_SIGNAL_PERIOD = 9
initial_investment = 100_000_000
# backup_amount_initial = 40_000_000

In [53]:
win_rate = 0.5711907018
loss_rate = 1 - win_rate
mean_profit = 0.3493357553
mean_loss = 0.1397008027
stop_loss = 0.08

In [54]:
def kelly_criterion(p, q, profit, loss):
    b = (profit * 160_000_000)/(loss * 160_000_000)
    f = (b*p - q)/b
    return f

In [55]:
f =kelly_criterion(win_rate,loss_rate,mean_profit,mean_loss)

In [56]:
high_ESG_group = ['CTD', 'DHG', 'DPM', 'FPT', 'GAS', 'MBB']
low_ESG_group = ['NVL', 'PNJ', 'REE', 'SBT', 'SSI', 'STB', 'VIC', 'VNM']
non_ESG_group = ['CII', 'CTG', 'EIB', 'GMD', 'HDB', 'HPG', 'MSN', 'MWG', 'ROS', 'SAB', 'TCB', 'VCB', 'VHM', 'VJC', 'VPB', 'VRE']


In [57]:
companies = ['VCB']

# Calculate indicators

In [58]:
def calculate_indicators(df):
    if df.empty:
        return df
    
    df['RSI'] = ta.momentum.RSIIndicator(df['close'], RSI_PERIOD).rsi()
    df['Previous_RSI'] = df['RSI'].shift(1)
    df['Previous_RSI'].fillna(0, inplace=True)
    macd = ta.trend.MACD(df['close'], window_slow=MACD_SLOW_PERIOD, window_fast=MACD_FAST_PERIOD, window_sign=MACD_SIGNAL_PERIOD)
    df['MACD'] = macd.macd()
    df['Signal_Line'] = macd.macd_signal()
    df['Previous_MACD'] = df['MACD'].shift(1)
    df['Previous_Signal_Line'] = df['Signal_Line'].shift(1)
    df['Previous_MACD'].fillna(0, inplace=True)
    df['Previous_Signal_Line'].fillna(0, inplace=True)

    return df

In [59]:
def macd_strategy(df):
    if df.empty:
        return df
    
    df['Signal'] = 0

    # Buy signals: RSI cross above 30 and MACD cross above Signal line
    df.loc[
        (df['Previous_MACD'] < df['Previous_Signal_Line']) &
        (df['MACD'] >= df['Signal_Line']) &
        (df['RSI'] > RSI_OVERSOLD), 'Signal'] = 1

    # Sell Signals: 
    df.loc[
        (df['RSI'] < RSI_OVERBOUGHT) &
        (df['Previous_MACD'] > df['Previous_Signal_Line']) &
        (df['MACD'] <= df['Signal_Line']), 'Signal'] = -1

    return df

In [60]:
def get_next_trading_day(date, trading_days):
    while date not in trading_days:
        date += BDay(1)
    return date

In [61]:
def calculate_monthly_rate(annual_rate, method='compounding'):
    """
    Converts an annual risk-free rate to a monthly rate.

    Parameters:
    annual_rate (float): The yearly risk-free rate as a decimal (e.g., 0.05 for 5%).
    method (str): The conversion method, either 'compounding' or 'simple'.

    Returns:
    float: The monthly risk-free rate.
    """
    if method == 'compounding':
        # Compounded monthly rate
        monthly_rate = (1 + annual_rate) ** (1 / 12) - 1
    elif method == 'simple':
        # Simple division approximation
        monthly_rate = annual_rate / 12
    else:
        raise ValueError("Method must be either 'compounding' or 'simple'")
    
    return monthly_rate

# Example usage
annual_rate = rf_2022  # For example, a 5% annual rate
rf_2022_monthly = calculate_monthly_rate(annual_rate, method='compounding')
monthly_rate_simple = calculate_monthly_rate(annual_rate, method='simple')

print(f"Monthly Rate (Compounding): {rf_2022_monthly:.6f}")
print(f"Monthly Rate (Simple): {monthly_rate_simple:.6f}")


Monthly Rate (Compounding): 0.002750
Monthly Rate (Simple): 0.002792


# Backtest

In [62]:
def simulate_investment(
    ticker, win_rate, loss_rate, mean_profit, mean_loss, 
    sell_fraction, start_date, end_date, f_star=1
):
    try:
        # Initialize trade counters and portfolio metrics
        number_of_buying_trades = 0
        number_of_selling_trades = 0
        cash = initial_investment
        holdings = 0
        portfolio_values = []

        # Load stock data and calculate indicators
        data = Vnstock().stock(symbol=ticker, source='TCBS').quote.history(start=start_date, end=end_date)
        data = data.set_index(pd.DatetimeIndex(data['time'].values))
        data = calculate_indicators(data)
        data = macd_strategy(data)

        trading_days = data.index
        buy_signals = data[data['Signal'] == 1].index
        sell_signals = data[data['Signal'] == -1].index

        pending_buy_shares = {}
        pending_sell_revenue = {}
        
        for i, current_date in enumerate(data.index):
            current_price = data['close'].iloc[i]

            # Handle pending T+2 settlements
            if current_date in pending_buy_shares:
                holdings += pending_buy_shares.pop(current_date)
            if current_date in pending_sell_revenue:
                cash += pending_sell_revenue.pop(current_date)

            # Avoid trades in January 2024
            if current_date.month == 1 and current_date.year == 2024:
                portfolio_values.append(cash + holdings * current_price)
                continue

            # Buy if there's a buy signal and cash allows
            if current_date in buy_signals:
                allocation = cash * f_star
                shares_to_buy = int(allocation // current_price)
                total_cost = shares_to_buy * current_price
                if shares_to_buy > 0 and cash >= total_cost:
                    cash -= total_cost
                    settlement_date = get_next_trading_day(current_date + BDay(2), trading_days)
                    pending_buy_shares[settlement_date] = pending_buy_shares.get(settlement_date, 0) + shares_to_buy
                    last_buy_price = current_price
                    number_of_buying_trades += 1

            # Sell if there's a sell signal and holdings allow
            if holdings > 0 and current_date in sell_signals:
                shares_to_sell = int(holdings * sell_fraction)
                revenue = shares_to_sell * current_price
                holdings -= shares_to_sell
                settlement_date = get_next_trading_day(current_date + BDay(2), trading_days)
                pending_sell_revenue[settlement_date] = pending_sell_revenue.get(settlement_date, 0) + revenue
                number_of_selling_trades += 1

            # Update portfolio value
            portfolio_values.append(cash + holdings * current_price)

        # Finalize portfolio values including pending settlements
        final_date = data.index[-1]
        while final_date <= data.index[-1] + BDay(2):
            if final_date in pending_buy_shares:
                holdings += pending_buy_shares.pop(final_date)
            if final_date in pending_sell_revenue:
                cash += pending_sell_revenue.pop(final_date)
            portfolio_values.append(cash + holdings * data['close'].iloc[-1])
            final_date += BDay(1)

        # Adjust portfolio values to match data index length
        if len(portfolio_values) > len(data.index):
            portfolio_values = portfolio_values[:len(data.index)]
        elif len(portfolio_values) < len(data.index):
            portfolio_values.extend([portfolio_values[-1]] * (len(data.index) - len(portfolio_values)))

        # Add portfolio values to data frame
        data['Portfolio_Value'] = portfolio_values
        data['Accumulated_Profit'] = data['Portfolio_Value'] - initial_investment
        data['Number_of_Buying_Trades'] = number_of_buying_trades
        data['Number_of_Selling_Trades'] = number_of_selling_trades
        data['Daily_Return'] = data['Portfolio_Value'].pct_change()

        # Calculate annual returns
        
        
        if start_date=='2022-01-01' and end_date=='2023-01-01':
            data['Month'] = data.index.month
            annual_data = data.groupby('Month').agg(
            Start_Value=('Portfolio_Value', 'first'),
            End_Value=('Portfolio_Value', 'last')
        )
        else:
            data['Year'] = data.index.year
            annual_data = data.groupby('Year').agg(
                Start_Value=('Portfolio_Value', 'first'),
                End_Value=('Portfolio_Value', 'last')
            )
        
        # Calculate Annual Return
        annual_data['Annual_Return'] = (annual_data['End_Value'] - annual_data['Start_Value']) / annual_data['Start_Value']
        
        # Merge back to main data frame if needed
        data = data.merge(annual_data[['Annual_Return']], left_on='Year' if 'Year' in data else 'Month', right_index=True, how='left')

        data['Cumulative_Max'] = data['Portfolio_Value'].cummax()  # Running max portfolio value
        data['Drawdown'] = (data['Portfolio_Value'] - data['Cumulative_Max']) / data['Cumulative_Max']
        max_drawdown = data['Drawdown'].min()  # Minimum drawdown value (most negative)

        data['Max_Drawdown'] = max_drawdown
        return data

    except Exception as e:
        print(f"Error occurred for {ticker}: {e}")
        return pd.DataFrame()


In [63]:
def calculate_sharpe_ratio(data, risk_free_rate=0.01):
    # Calculate daily returns from the Portfolio Value
    daily_returns = data['Annual_Return'].dropna()

    # Calculate average return and standard deviation of returns
    average_return = daily_returns.mean()
    std_deviation = daily_returns.std()

    # Calculate the Sharpe Ratio
    sharpe_ratio = (average_return - risk_free_rate) / std_deviation

    return sharpe_ratio

In [64]:
def calculate_sortino_ratio(data,rf=0.01):  # Target return can be set to risk-free rate
    # Calculate daily returns from the Portfolio Value
    daily_returns = data['Annual_Return'].dropna()

    # Calculate average return
    average_return = daily_returns.mean()

    # Calculate downside returns (returns below the target return)
    downside_returns = daily_returns[daily_returns < 0]

    # Calculate downside deviation
    downside_deviation = np.sqrt((downside_returns**2).mean()) if len(downside_returns) > 0 else np.nan

    # Calculate the Sortino Ratio
    sortino_ratio = (average_return - rf) / downside_deviation if downside_deviation > 0 else np.nan

    return sortino_ratio

thêm phần điều chỉnh kelly trong code phía dưới

In [86]:
def backtest_multiple_companies(companies_vn30, win_rate, loss_rate, mean_profit, mean_loss, sell_fraction, start_date, end_date, rf):
    results = []
    for company in companies_vn30:
        result = simulate_investment(company, win_rate, loss_rate, mean_profit, mean_loss, sell_fraction, start_date=start_date, end_date=end_date, f_star=1)
        if not result.empty:
            # Calculate the Sharpe Ratio for the result
            sharpe_ratio = calculate_sharpe_ratio(result,risk_free_rate=rf)
            results.append({
                'Company': company,
                'Final Portfolio Value': result['Portfolio_Value'].iloc[-1],
                'Total Profit': result['Accumulated_Profit'].iloc[-1],
                'Rate of Return': result['Accumulated_Profit'].iloc[-1] / initial_investment * 100,
                'Number of Buying Trades': result['Number_of_Buying_Trades'].iloc[-1],
                'Number of Selling Trades': result['Number_of_Selling_Trades'].iloc[-1],
                'Sharpe Ratio': sharpe_ratio,
                'Sortino Ratio': calculate_sortino_ratio(result,rf=rf),
                'MDD': result['Max_Drawdown'].min(),
            })
    return pd.DataFrame(results)

# Kelly 2019-2024

In [87]:
sell_fraction = 1 #kelly_criterion(win_rate, loss_rate, mean_profit, mean_loss)
results_df = backtest_multiple_companies(high_ESG_group, win_rate, loss_rate, mean_profit, mean_loss, sell_fraction, start_date='2019-01-01', end_date='2024-01-01', rf=rf_cal)
high_esg = pd.DataFrame(results_df)
print(high_esg)
average_rate_of_return = results_df['Rate of Return'].mean()
average_profit = results_df[results_df['Rate of Return'] > 0]['Rate of Return'].mean()
average_loss = results_df[results_df['Rate of Return'] < 0]['Rate of Return'].mean()
print("Average Rate of Return for 30 companies:", average_rate_of_return)
print(f'avg_ror: {average_rate_of_return}, avg_profit: {average_profit}, avg loss: {average_loss}','avg sharpe:',results_df['Sharpe Ratio'].mean(),'avg sortino:',results_df['Sortino Ratio'].mean())

  Company  Final Portfolio Value  Total Profit  Rate of Return  \
0     CTD           1.783510e+08   78350964.04       78.350964   
1     DHG           1.080992e+08    8099224.03        8.099224   
2     DPM           1.098032e+08    9803232.94        9.803233   
3     FPT           1.332869e+08   33286881.43       33.286881   
4     GAS           9.811326e+07   -1886744.36       -1.886744   
5     MBB           9.805959e+07   -1940413.00       -1.940413   

   Number of Buying Trades  Number of Selling Trades  Sharpe Ratio  \
0                        7                         6     15.093093   
1                        7                         6      9.411268   
2                        8                         7     10.512751   
3                        8                         7     14.206281   
4                        8                         7    -42.950432   
5                        8                         7    -42.198062   

   Sortino Ratio  MDD  
0            NaN -1.0 

In [88]:
sell_fraction = 1 #kelly_criterion(win_rate, loss_rate, mean_profit, mean_loss)
low_esg_kelly = backtest_multiple_companies(low_ESG_group, win_rate, loss_rate, mean_profit, mean_loss, sell_fraction, start_date='2019-01-01', end_date='2024-01-04', rf=rf_cal)
low_esg_kelly = pd.DataFrame(low_esg_kelly)
print(low_esg_kelly)
average_rate_of_return = low_esg_kelly['Rate of Return'].mean()
average_profit = low_esg_kelly[low_esg_kelly['Rate of Return'] > 0]['Rate of Return'].mean()
average_loss = low_esg_kelly[low_esg_kelly['Rate of Return'] < 0]['Rate of Return'].mean()
print("Average Rate of Return for 30 companies:", average_rate_of_return)
print(f'avg_ror: {average_rate_of_return}, avg_profit: {average_profit}, avg loss: {average_loss}')

  Company  Final Portfolio Value  Total Profit  Rate of Return  \
0     NVL           1.500353e+08   50035345.95       50.035346   
1     PNJ           9.731911e+07   -2680888.84       -2.680889   
2     REE           9.740726e+07   -2592742.94       -2.592743   
3     SBT           9.173568e+07   -8264318.80       -8.264319   
4     SSI           1.426331e+08   42633133.04       42.633133   
5     STB           9.869448e+07   -1305515.45       -1.305515   
6     VIC           1.053460e+08    5346027.10        5.346027   
7     VNM           9.679916e+07   -3200835.61       -3.200836   

   Number of Buying Trades  Number of Selling Trades  Sharpe Ratio  \
0                        8                         8      8.460996   
1                        9                         8    -71.724700   
2                        8                         8    -20.530525   
3                       10                         9    -11.775678   
4                        9                         8   

In [89]:
sell_fraction = 1 #kelly_criterion(win_rate, loss_rate, mean_profit, mean_loss)
non_esg_kelly = backtest_multiple_companies(non_ESG_group, win_rate, loss_rate, mean_profit, mean_loss, sell_fraction, start_date='2019-01-01', end_date='2024-01-04', rf=rf_cal)
non_esg_kelly = pd.DataFrame(non_esg_kelly)
print(non_esg_kelly)
average_rate_of_return = non_esg_kelly['Rate of Return'].mean()
average_profit = non_esg_kelly[non_esg_kelly['Rate of Return'] > 0]['Rate of Return'].mean()
average_loss = non_esg_kelly[non_esg_kelly['Rate of Return'] < 0]['Rate of Return'].mean()
print("Average Rate of Return for 30 companies:", average_rate_of_return)
print(f'avg_ror: {average_rate_of_return}, avg_profit: {average_profit}, avg loss: {average_loss}')

   Company  Final Portfolio Value  Total Profit  Rate of Return  \
0      CII           1.256725e+08   25672522.77       25.672523   
1      CTG           9.468683e+07   -5313170.57       -5.313171   
2      EIB           1.202521e+08   20252145.79       20.252146   
3      GMD           1.114129e+08   11412948.32       11.412948   
4      HDB           1.102215e+08   10221475.18       10.221475   
5      HPG           1.112884e+08   11288352.69       11.288353   
6      MSN           9.232978e+07   -7670222.00       -7.670222   
7      MWG           1.200333e+08   20033326.72       20.033327   
8      ROS           1.677895e+08   67789537.55       67.789538   
9      SAB           8.400447e+07  -15995528.98      -15.995529   
10     TCB           9.651521e+07   -3484791.67       -3.484792   
11     VCB           9.244866e+07   -7551337.37       -7.551337   
12     VHM           1.121656e+08   12165595.10       12.165595   
13     VJC           9.741252e+07   -2587481.80       -2.58748

In [90]:
high_esg['ESG'] ='High'
low_esg_kelly['ESG'] = 'Low'
non_esg_kelly['ESG'] = 'Non'

In [91]:
merged = pd.concat([high_esg, low_esg_kelly, non_esg_kelly]) 
merged['period'] = '2019-2024' 

In [92]:
merged

,Company,Final Portfolio Value,Total Profit,Rate of Return,Number of Buying Trades,Number of Selling Trades,Sharpe Ratio,Sortino Ratio,MDD,ESG,period
0,CTD,1.783510e+08,78350964.04,78.350964,7,6,15.093093,NaN,-1.0,High,2019-2024
1,DHG,1.080992e+08,8099224.03,8.099224,7,6,9.411268,NaN,-1.0,High,2019-2024
2,DPM,1.098032e+08,9803232.94,9.803233,8,7,10.512751,NaN,-1.0,High,2019-2024
3,FPT,1.332869e+08,33286881.43,33.286881,8,7,14.206281,NaN,-1.0,High,2019-2024
4,GAS,9.811326e+07,-1886744.36,-1.886744,8,7,-42.950432,-2.716424,-1.0,High,2019-2024
5,MBB,9.805959e+07,-1940413.00,-1.940413,8,7,-42.198062,-2.668840,-1.0,High,2019-2024
0,NVL,1.500353e+08,50035345.95,50.035346,8,8,8.460996,NaN,-1.0,Low,2019-2024
1,PNJ,9.731911e+07,-2680888.84,-2.680889,9,8,-71.724700,-4.000938,-1.0,Low,2019-2024
2,REE,9.740726e+07,-2592742.94,-2.592743,8,8,-20.530525,-2.239956,-1.0,Low,2019-2024
3,SBT,9.173568e+07,-8264318.80,-8.264319,10,9,-11.775678,-1.336733,-1.0,Low,2019-2024


# Kelly 2022

In [93]:
sell_fraction = 1 #kelly_criterion(win_rate, loss_rate, mean_profit, mean_loss)
high_esg_2022 = backtest_multiple_companies(high_ESG_group, win_rate, loss_rate, mean_profit, mean_loss, sell_fraction, start_date='2022-01-01', end_date='2023-01-01', rf=rf_2022_monthly)
high_esg_2022 = pd.DataFrame(high_esg_2022)
print(high_esg_2022)
average_rate_of_return = high_esg_2022['Rate of Return'].mean()
average_profit = high_esg_2022[high_esg_2022['Rate of Return'] > 0]['Rate of Return'].mean()
average_loss = high_esg_2022[high_esg_2022['Rate of Return'] < 0]['Rate of Return'].mean()
print("Average Rate of Return for 30 companies:", average_rate_of_return)
print(f'avg_ror: {average_rate_of_return}, avg_profit: {average_profit}, avg loss: {average_loss}')
high_esg_2022['ESG'] ='High'

  Company  Final Portfolio Value  Total Profit  Rate of Return  \
0     CTD           1.006370e+08     636981.85        0.636982   
1     DHG           8.473025e+07  -15269747.53      -15.269748   
2     DPM           8.727333e+07  -12726670.70      -12.726671   
3     FPT           1.223621e+08   22362121.55       22.362122   
4     GAS           1.042441e+08    4244080.41        4.244080   
5     MBB           1.025689e+08    2568867.35        2.568867   

   Number of Buying Trades  Number of Selling Trades  Sharpe Ratio  \
0                        8                         8      0.317673   
1                        7                         6      0.294294   
2                        8                         8      0.367469   
3                        5                         5     -0.215260   
4                        7                         7      0.097233   
5                        6                         6     -0.448815   

   Sortino Ratio  MDD  
0   2.073537e+09 -1.0 

In [94]:
sell_fraction = 1 #kelly_criterion(win_rate, loss_rate, mean_profit, mean_loss)
low_esg_kelly_2022 = backtest_multiple_companies(low_ESG_group, win_rate, loss_rate, mean_profit, mean_loss, sell_fraction, start_date='2022-01-01', end_date='2023-01-01', rf=rf_2022_monthly)
low_esg_kelly_2022 = pd.DataFrame(low_esg_kelly_2022)
print(low_esg_kelly_2022)
average_rate_of_return = low_esg_kelly_2022['Rate of Return'].mean()
average_profit = low_esg_kelly_2022[low_esg_kelly_2022['Rate of Return'] > 0]['Rate of Return'].mean()
average_loss = low_esg_kelly_2022[low_esg_kelly_2022['Rate of Return'] < 0]['Rate of Return'].mean()
print("Average Rate of Return for 30 companies:", average_rate_of_return)
print(f'avg_ror: {average_rate_of_return}, avg_profit: {average_profit}, avg loss: {average_loss}')
low_esg_kelly_2022['ESG'] = 'Low'

Error occurred for PNJ: Out of bounds nanosecond timestamp: 9223545600000000000
  Company  Final Portfolio Value  Total Profit  Rate of Return  \
0     NVL           1.058145e+08    5814451.80        5.814452   
1     REE           9.013103e+07   -9868973.26       -9.868973   
2     SBT           1.093378e+08    9337817.87        9.337818   
3     SSI           9.148210e+07   -8517899.14       -8.517899   
4     STB           9.659293e+07   -3407067.45       -3.407067   
5     VIC           9.455290e+07   -5447101.10       -5.447101   
6     VNM           1.003832e+08     383151.22        0.383151   

   Number of Buying Trades  Number of Selling Trades  Sharpe Ratio  \
0                        6                         6      0.302219   
1                        9                         9      0.317673   
2                        8                         7     -0.230383   
3                        7                         7      0.310008   
4                        4               

In [95]:
sell_fraction = 1 #kelly_criterion(win_rate, loss_rate, mean_profit, mean_loss)
non_esg_kelly_2022 = backtest_multiple_companies(non_ESG_group, win_rate, loss_rate, mean_profit, mean_loss, sell_fraction, start_date='2022-01-01', end_date='2023-01-01', rf=rf_2022_monthly)
non_esg_kelly_2022 = pd.DataFrame(non_esg_kelly_2022)
print(non_esg_kelly_2022)
average_rate_of_return = non_esg_kelly_2022['Rate of Return'].mean()
average_profit = non_esg_kelly_2022[non_esg_kelly_2022['Rate of Return'] > 0]['Rate of Return'].mean()
average_loss = non_esg_kelly_2022[non_esg_kelly_2022['Rate of Return'] < 0]['Rate of Return'].mean()
print("Average Rate of Return for 30 companies:", average_rate_of_return)
print(f'avg_ror: {average_rate_of_return}, avg_profit: {average_profit}, avg loss: {average_loss}')
non_esg_kelly_2022['ESG'] = 'Non'

Error occurred for SAB: Out of bounds nanosecond timestamp: 9223545600000000000
Error occurred for VPB: Out of bounds nanosecond timestamp: 9223545600000000000
   Company  Final Portfolio Value  Total Profit  Rate of Return  \
0      CII           8.125454e+07  -18745463.00      -18.745463   
1      CTG           1.243028e+08   24302785.54       24.302786   
2      EIB           1.284029e+08   28402852.77       28.402853   
3      GMD           7.783641e+07  -22163585.55      -22.163586   
4      HDB           9.484021e+07   -5159785.69       -5.159786   
5      HPG           1.005819e+08     581892.45        0.581892   
6      MSN           1.162306e+08   16230557.50       16.230557   
7      MWG           1.209428e+08   20942812.48       20.942812   
8      ROS           1.677895e+08   67789537.55       67.789538   
9      TCB           1.072849e+08    7284916.70        7.284917   
10     VCB           8.792756e+07  -12072436.04      -12.072436   
11     VHM           8.059357e+07  -

In [96]:
merged_2022 = pd.concat([high_esg_2022, low_esg_kelly_2022, non_esg_kelly_2022])
merged_2022['period'] = '2022-2023'
merged_2022

,Company,Final Portfolio Value,Total Profit,Rate of Return,Number of Buying Trades,Number of Selling Trades,Sharpe Ratio,Sortino Ratio,MDD,ESG,period
0,CTD,1.006370e+08,636981.85,0.636982,8,8,0.317673,2.073537e+09,-1.0,High,2022-2023
1,DHG,8.473025e+07,-15269747.53,-15.269748,7,6,0.294294,4.379287e+06,-1.0,High,2022-2023
2,DPM,8.727333e+07,-12726670.70,-12.726671,8,8,0.367469,2.394750e+06,-1.0,High,2022-2023
3,FPT,1.223621e+08,22362121.55,22.362122,5,5,-0.215260,-1.294336e-01,-1.0,High,2022-2023
4,GAS,1.042441e+08,4244080.41,4.244080,7,7,0.097233,8.201866e-02,-1.0,High,2022-2023
5,MBB,1.025689e+08,2568867.35,2.568867,6,6,-0.448815,-2.709925e-01,-1.0,High,2022-2023
0,NVL,1.058145e+08,5814451.80,5.814452,6,6,0.302219,5.079105e+05,-1.0,Low,2022-2023
1,REE,9.013103e+07,-9868973.26,-9.868973,9,9,0.317673,3.462456e+05,-1.0,Low,2022-2023
2,SBT,1.093378e+08,9337817.87,9.337818,8,7,-0.230383,-1.291500e-01,-1.0,Low,2022-2023
3,SSI,9.148210e+07,-8517899.14,-8.517899,7,7,0.310008,3.152443e+06,-1.0,Low,2022-2023


# Gộp 2 cái

In [97]:
merged_all = pd.concat([merged, merged_2022])

In [98]:
merged_all= merged_all.reset_index()

In [99]:
merged_all

,index,Company,Final Portfolio Value,Total Profit,Rate of Return,Number of Buying Trades,Number of Selling Trades,Sharpe Ratio,Sortino Ratio,MDD,ESG,period
0,0,CTD,1.783510e+08,78350964.04,78.350964,7,6,15.093093,NaN,-1.0,High,2019-2024
1,1,DHG,1.080992e+08,8099224.03,8.099224,7,6,9.411268,NaN,-1.0,High,2019-2024
2,2,DPM,1.098032e+08,9803232.94,9.803233,8,7,10.512751,NaN,-1.0,High,2019-2024
3,3,FPT,1.332869e+08,33286881.43,33.286881,8,7,14.206281,NaN,-1.0,High,2019-2024
4,4,GAS,9.811326e+07,-1886744.36,-1.886744,8,7,-42.950432,-2.716424e+00,-1.0,High,2019-2024
5,5,MBB,9.805959e+07,-1940413.00,-1.940413,8,7,-42.198062,-2.668840e+00,-1.0,High,2019-2024
6,0,NVL,1.500353e+08,50035345.95,50.035346,8,8,8.460996,NaN,-1.0,Low,2019-2024
7,1,PNJ,9.731911e+07,-2680888.84,-2.680889,9,8,-71.724700,-4.000938e+00,-1.0,Low,2019-2024
8,2,REE,9.740726e+07,-2592742.94,-2.592743,8,8,-20.530525,-2.239956e+00,-1.0,Low,2019-2024
9,3,SBT,9.173568e+07,-8264318.80,-8.264319,10,9,-11.775678,-1.336733e+00,-1.0,Low,2019-2024


In [100]:
merged_all['total_trades']= merged_all['Number of Buying Trades'] + merged_all['Number of Selling Trades']
merged_all.to_csv('macd kelly.csv', index=False)

In [101]:
A= merged_all.groupby(['ESG','period'])[['Rate of Return', 'Sharpe Ratio', 'Sortino Ratio', 'total_trades', 'MDD']].mean()
A

Rate of Return  Sharpe Ratio  Sortino Ratio  total_trades  MDD
ESG  period                                                                   
High 2019-2024       20.952191     -5.987517  -2.692632e+00     14.333333 -1.0
     2022-2023        0.302605      0.068766   3.467185e+08     13.500000 -1.0
Low  2019-2024        9.996276    -15.197633  -2.017229e+00     16.500000 -1.0
     2022-2023       -1.672231      0.028973   1.011764e+06     13.571429 -1.0
Non  2019-2024        8.330674     -4.095007  -1.076887e+00     13.875000 -1.0
     2022-2023        6.256444      0.015538   4.372805e+06     12.071429 -1.0

In [102]:
merged_all.groupby(['ESG','period'])[['Rate of Return', 'Sharpe Ratio', 'Sortino Ratio', 'total_trades', 'MDD']].mean()


Rate of Return  Sharpe Ratio  Sortino Ratio  total_trades  MDD
ESG  period                                                                   
High 2019-2024       20.952191     -5.987517  -2.692632e+00     14.333333 -1.0
     2022-2023        0.302605      0.068766   3.467185e+08     13.500000 -1.0
Low  2019-2024        9.996276    -15.197633  -2.017229e+00     16.500000 -1.0
     2022-2023       -1.672231      0.028973   1.011764e+06     13.571429 -1.0
Non  2019-2024        8.330674     -4.095007  -1.076887e+00     13.875000 -1.0
     2022-2023        6.256444      0.015538   4.372805e+06     12.071429 -1.0

In [103]:
kelly = pd.read_csv('macd kelly.csv')
no_kelly = pd.read_csv('macd no kelly.csv')

In [104]:
kelly['Kelly'] = 'Yes'
no_kelly['Kelly'] = 'No'
double_merged = pd.concat([kelly, no_kelly])
double_merged.to_excel('macd.xlsx', index=False)

In [107]:
B= double_merged.groupby(['ESG','period','Kelly'])[['Rate of Return', 'Sharpe Ratio', 'Sortino Ratio', 'total_trades', 'MDD']].mean()
B.to_excel('double_merged_macd.xlsx')

In [108]:
B

Rate of Return  Sharpe Ratio  Sortino Ratio  \
ESG  period    Kelly                                                
High 2019-2024 No          20.952191     -5.987517  -2.692632e+00   
               Yes         20.952191     -5.987517  -2.692632e+00   
     2022-2023 No           0.302605      0.068766   3.467185e+08   
               Yes          0.302605      0.068766   3.467185e+08   
Low  2019-2024 No           9.996276    -15.197633  -2.017229e+00   
               Yes          9.996276    -15.197633  -2.017229e+00   
     2022-2023 No          -1.672231      0.028973   1.011764e+06   
               Yes         -1.672231      0.028973   1.011764e+06   
Non  2019-2024 No           8.330674     -4.095007  -1.076887e+00   
               Yes          8.330674     -4.095007  -1.076887e+00   
     2022-2023 No           6.256444      0.015538   4.372805e+06   
               Yes          6.256444      0.015538   4.372805e+06   

                      total_trades  MDD  
ESG  period    Kelly                     
High 2019-2024 No        14.333333 -1.0  
               Yes       14.333333 -1.0  
     2022-2023 No        13.500000 -1.0  
               Yes       13.500000 -1.0  
Low  2019-2024 No        16.500000 -1.0  
               Yes       16.500000 -1.0  
     2022-2023 No        13.571429 -1.0  
               Yes       13.571429 -1.0  
Non  2019-2024 No        13.875000 -1.0  
               Yes       13.875000 -1.0  
     2022-2023 No        12.071429 -1.0  
               Yes       12.071429 -1.0